In [7]:
%%time
reload_data = False

import joblib, time, os, copy, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision

#enable it when running in google cloud, to upload/download file from/to VM to/from google cloud bucket
from google.cloud import storage

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#turn on cpu for troubleshooting as gpu doesnt throw proper error message, runing in cpu shows more specific error message
#device = torch.device("cpu") 
print(device)

bucket_root_path = "weicheng30417"
project_data_folder = "data/breakfast-img-data/"

cuda:0
CPU times: user 240 ms, sys: 2.61 s, total: 2.85 s
Wall time: 3.11 s


In [2]:
def download_all_data_from_bucket():
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    file_id = project_data_folder
    blobs=bucket.list_blobs(prefix=file_id)
    for blob in blobs:        
        file_names = blob.name.split("/")
        if len(file_names) == 3:
            if file_names[2] != "":
                file_name = file_names[2]            
                blob.download_to_filename(file_name)
                print("Download from {0} to local {1}".format(blob.name, file_name))
                

def download_file_from_bucket(file):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    butcketFile = project_data_folder + file
    blob = bucket.blob(butcketFile)
    blob.download_to_filename(file)
    print("Download from {0} to local {1}".format(butcketFile, file))
    
def upload_all_files():    
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    
    directory = os.getcwd()
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv") or filename.endswith(".ipynb") or filename.endswith(".model"):             
            butcketFile = project_data_folder + file
            blob = bucket.blob(butcketFile)
            blob.upload_from_filename(filename)
            print("Upload from local {0} to {1}".format(filename, butcketFile))
            
def upload_files(files):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    for file in files:
        butcketFile = project_data_folder + file
        blob = bucket.blob(butcketFile)
        blob.upload_from_filename(file)
        print("Upload from local {0} to {1}".format(file, butcketFile))

def flatten(ls):
     for item in ls:
            for child in list(item):
                yield child

def get_segment_positions(x):
    ps = x.segment.split()
    ls = []
    for i in range(len(ps)-1):
        if i == 0:
            ls.append([int(ps[i]), int(ps[i+1])])
        else:
            ls.append([int(ps[i])+1, int(ps[i+1])])        
    return ls

def get_segment_features(x):
    ls = []
    for rg in x.positions:
        ls.append(x.feature[rg[0]:rg[1]])            
    return ls

def splitDataFrameList(df,target_column):
    def splitListToRows(row,row_accumulator,target_column):
        split_row = row[target_column]
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column))
    new_df = pd.DataFrame(new_rows)
    return new_df

def extract_or_pad(x):    
    target = torch.zeros([400, 400], dtype=torch.float64)
    w = x.shape[0]
    if x.shape[0] > 400:
        target = x[:400, :400]
    else:
        target[:w, :400] = x
        
    return target.view(1, 400, 400)
                
def get_train_data():
    training_segment = pd.read_csv('training_segment.txt', header=None, names = ['segment'])
    training_segment['feature'] = joblib.load('train_feature.joblib')
    training_segment['positions'] = training_segment.apply(lambda x: get_segment_positions(x), axis=1)
    training_segment['feature'] = training_segment.apply(lambda x: get_segment_features(x), axis=1)
    training_segment = splitDataFrameList(training_segment, 'feature')
    training_segment['label'] = list(flatten(joblib.load('train_label.joblib')))
    training_segment = training_segment.drop(['segment','positions'], axis = 1)
    return training_segment

def get_test_data():
    test_segment = pd.read_csv('test_segment.txt', header=None, names = ['segment'])
    test_segment['feature'] = joblib.load('test_feature.joblib')
    test_segment['positions'] = test_segment.apply(lambda x: get_segment_positions(x), axis=1)
    test_segment['feature'] = test_segment.apply(lambda x: get_segment_features(x), axis=1)
    test_segment = splitDataFrameList(test_segment, 'feature')
    test_segment['ID'] = test_segment.index
    test_segment = test_segment.drop(['segment','positions'], axis = 1)
    return test_segment

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        ms = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)           

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)            
            print('{} Loss: {:.4f}, {} Acc: {:.4f}'.format(phase, epoch_loss, phase, epoch_acc))
        
        time_taken = str(datetime.timedelta(seconds=time.time() - ms))
        print('time taken: {}'.format(time_taken))
            

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [3]:
%%time
if reload_data:
    download_all_data_from_bucket()

Download from data/breakfast-img-data/model_baseline.model to local model_baseline.model
Download from data/breakfast-img-data/submission_1.0_max.csv to local submission_1.0_max.csv
Download from data/breakfast-img-data/submission_1.0_mean.csv to local submission_1.0_mean.csv
Download from data/breakfast-img-data/test_feature.joblib to local test_feature.joblib
Download from data/breakfast-img-data/test_segment.txt to local test_segment.txt
Download from data/breakfast-img-data/train_feature.joblib to local train_feature.joblib
Download from data/breakfast-img-data/train_label.joblib to local train_label.joblib
Download from data/breakfast-img-data/training_segment.txt to local training_segment.txt
CPU times: user 59 s, sys: 17.5 s, total: 1min 16s
Wall time: 2min 53s


In [4]:
%%time
train_df = get_train_data()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7075 entries, 0 to 7074
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  7075 non-null   object
 1   label    7075 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 110.7+ KB
CPU times: user 2.45 s, sys: 9.89 s, total: 12.3 s
Wall time: 24.4 s


In [5]:
%%time
def extract_or_pad(x):    
    target = torch.zeros([400, 400], dtype=torch.float64)
    w = x.shape[0]
    if x.shape[0] > 400:
        target = x[:400, :400]
    else:
        target[:w, :400] = x
        
    return target.view(1, 400, 400)
    #return target

train_data = train_df.sample(frac=1.0)
train_data['label'] = train_data.apply(lambda x : x["label"] - 1, axis = 1)
train_data['feature'] = train_data.apply(lambda x : extract_or_pad(x.feature), axis = 1)
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7075 entries, 1631 to 1691
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  7075 non-null   object
 1   label    7075 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 165.8+ KB
None
CPU times: user 4.61 s, sys: 3.46 s, total: 8.07 s
Wall time: 1.7 s


In [6]:
%%time
num_classes = 47
batch_size = 8
num_epochs = 10
feature_extract = True

train_feature = torch.stack(train_data['feature'].tolist())
train_label = torch.tensor(train_data['label'].values.astype(np.long))

train_dataset = torch.utils.data.TensorDataset(train_feature, train_label)

train_size = int(0.8 * len(train_data))
val_size = int(0.1 * len(train_data))
test_size = len(train_data) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

dataloaders_dict = {'train':train_loader, 'val':test_loader}
print("len(train_loader.dataset) = ", len(train_loader.dataset))
print("len(val_loader.dataset) = ", len(val_loader.dataset))
print("len(test_loader.dataset) = ", len(test_loader.dataset))

len(train_loader.dataset) =  5660
len(val_loader.dataset) =  707
len(test_loader.dataset) =  708
CPU times: user 2.32 s, sys: 3.9 s, total: 6.23 s
Wall time: 5.49 s


In [3]:
from torch.nn import *
from torch.nn.functional import *

class MyNet(Module):
    def __init__(self, num_classes):
        super(MyNet, self).__init__()
        self.conv1 = Conv2d(in_channels=1, out_channels=batch_size, kernel_size=3, padding=1)
        self.conv1_bn = BatchNorm2d(num_features=8)
        self.conv2 = Conv2d(in_channels=8, out_channels=32, kernel_size=3, padding=1)
        self.conv2_bn = BatchNorm2d(num_features=32)
        self.conv3 = Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv4 = Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv5 = Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv5_bn = BatchNorm2d(num_features=256)
        self.pool = MaxPool2d(2, 2)
        self.drop1 = Dropout(0.5)
        self.linear1 = Linear(in_features=512, out_features=128)
        self.lstm1 = LSTM(input_size=400, hidden_size=128, num_layers=2, bidirectional=True, dropout = 0.5)
        self.linear2 = Linear(in_features=256, out_features=num_classes)
    
    def forward(self, x):
        x = self.pool(relu(self.conv1_bn(self.conv1(x))))        
        x = self.pool(relu(self.conv2_bn(self.conv2(x))))        
        x = relu(self.conv3(x))        
        x = relu(self.conv4(x))        
        x = self.pool(relu(self.conv5_bn(self.conv5(x))))
        x = x.view(-1, 512)        
        x = relu(self.linear1(x))        
        x = self.drop1(x) 
        x = x.view(-1, 400, 400)        
        x, _ = self.lstm1(x)
        #x = x.contiguous().view(-1, 128)        
        x = self.drop1(x) 
        x = log_softmax(self.linear2(x))
        x = x[:,-1]
        return x

In [7]:
%%time
model_ft = MyNet(num_classes).double()
print(model_ft)
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

#optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)  
optimizer_ft = torch.optim.Adam(params_to_update, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
criterion = torch.nn.CrossEntropyLoss()
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=False)

cuda:0
MyNet(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.5, inplace=False)
  (linear1): Linear(in_features=512, out_features=128, bias=True)
  (lstm1): LSTM(400, 128, num_layers=2, dropout=0.5, bidirectional=True)
  (linear2): Linear(in_features=256, out_features=47, 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


train Loss: 3.6893, train Acc: 0.0482
val Loss: 3.5198, val Acc: 0.0692
time taken: 0:17:27.497043
Epoch 1/9
----------
train Loss: 3.5832, train Acc: 0.0507
val Loss: 3.4749, val Acc: 0.0692
time taken: 0:17:27.073957
Epoch 2/9
----------
train Loss: 3.5201, train Acc: 0.0659
val Loss: 3.4467, val Acc: 0.0749
time taken: 0:17:26.747163
Epoch 3/9
----------
train Loss: 3.4747, train Acc: 0.0758
val Loss: 3.4335, val Acc: 0.0904
time taken: 0:17:26.911955
Epoch 4/9
----------
train Loss: 3.4581, train Acc: 0.0832
val Loss: 3.3946, val Acc: 0.0862
time taken: 0:17:26.732821
Epoch 5/9
----------
train Loss: 3.4113, train Acc: 0.0906
val Loss: 3.3709, val Acc: 0.0932
time taken: 0:17:26.983264
Epoch 6/9
----------
train Loss: 3.3796, train Acc: 0.0970
val Loss: 3.3339, val Acc: 0.0960
time taken: 0:17:26.670760
Epoch 7/9
----------
train Loss: 3.3446, train Acc: 0.1088
val Loss: 3.3132, val Acc: 0.0975
time taken: 0:17:26.999421
Epoch 8/9
----------
train Loss: 3.3328, train Acc: 0.1099
va

In [8]:
%%time
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test data: {0}'.format(100 * correct / total))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of test data: 10.169491525423728
CPU times: user 40.6 s, sys: 15.1 s, total: 55.7 s
Wall time: 55.7 s


In [9]:
%%time
model_name = "model_lstm.model" #renamed to model_2.0.model
torch.save(model_ft.state_dict(), model_name)
upload_files([model_name])

Upload from local model_lstm.model to data/breakfast-img-data/model_lstm.model
CPU times: user 36 ms, sys: 24 ms, total: 60 ms
Wall time: 2.49 s


In [8]:
%%time
batch_size = 8
model_name = "model_2.0.model"
model = MyNet(47).double().to(device)
model.load_state_dict(torch.load(model_name, map_location=device)) 

CPU times: user 3.76 s, sys: 1.19 s, total: 4.95 s
Wall time: 5.52 s


<All keys matched successfully>

In [32]:
%%time
test_df = get_test_data()
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  1284 non-null   object
 1   ID       1284 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.2+ KB
None
CPU times: user 336 ms, sys: 940 ms, total: 1.28 s
Wall time: 1.27 s


In [33]:
%%time
test_data = test_df.sample(frac = 1.0)
test_data['feature'] = test_data.apply(lambda x : extract_or_pad(x.feature), axis = 1)
print(test_data.info())

test_feature = torch.stack(test_data['feature'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        result = np.append(result, predicted.cpu())

test_df['Category'] = np.transpose(result).astype('int') 
test_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284 entries, 922 to 211
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  1284 non-null   object
 1   ID       1284 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 30.1+ KB
None


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


CPU times: user 1min 17s, sys: 29.3 s, total: 1min 47s
Wall time: 1min 45s


,feature,ID,Category
0,"[[tensor(-15.8486, dtype=torch.float64), tenso...",0,13
1,"[[tensor(-16.9285, dtype=torch.float64), tenso...",1,1
2,"[[tensor(-8.6689, dtype=torch.float64), tensor...",2,1
3,"[[tensor(-16.0939, dtype=torch.float64), tenso...",3,46
4,"[[tensor(-16.0982, dtype=torch.float64), tenso...",4,10
...,...,...,...
1279,"[[tensor(-7.8880, dtype=torch.float64), tensor...",1279,1
1280,"[[tensor(-8.6625, dtype=torch.float64), tensor...",1280,32
1281,"[[tensor(-7.1023, dtype=torch.float64), tensor...",1281,13
1282,"[[tensor(-11.9272, dtype=torch.float64), tenso...",1282,30


In [34]:
%%time
test_df['Category'] = test_df.apply(lambda x : x.Category + 1, axis = 1)
test_df = test_df[['ID','Category']]
test_df

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 22.3 ms


,ID,Category
0,0,14
1,1,2
2,2,2
3,3,47
4,4,11
...,...,...
1279,1279,2
1280,1280,33
1281,1281,14
1282,1282,31


In [35]:
len(test_df.Category.unique())

43

In [36]:
submission_name = "submission_2.0_max.csv"
test_df.to_csv(submission_name, index=False)

In [37]:
upload_files([submission_name])

Upload from local submission_2.0_max.csv to data/breakfast-img-data/submission_2.0_max.csv


In [22]:
%%time
test_df = get_test_data()
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  1284 non-null   object
 1   ID       1284 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.2+ KB
None
CPU times: user 336 ms, sys: 1.42 s, total: 1.76 s
Wall time: 1.76 s


In [24]:
%%time
test_data = test_df.sample(frac = 1.0)
test_data['feature'] = test_data.apply(lambda x : extract_or_pad(x.feature), axis = 1)
print(test_data.info())

test_feature = torch.stack(test_data['feature'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0]            
        inputs = inputs.to(device)    
        outputs = model(inputs)  
        prop = torch.nn.functional.softmax(outputs).cpu().tolist()
        result = np.append(result, prop)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284 entries, 502 to 1164
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  1284 non-null   object
 1   ID       1284 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 30.1+ KB
None


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


CPU times: user 1min 16s, sys: 29.2 s, total: 1min 46s
Wall time: 1min 44s


In [25]:
%%time
test_result = pd.DataFrame(result.reshape(-1,47))
test_result['ID'] = test_data['ID'].values
test_data_mean = test_result.groupby('ID')[list(range(47))].agg(['mean'])
test_data_mean.columns = list(range(47))
test_data_mean["Category"] = test_data_mean.idxmax(axis = 1)
test_data_mean['Category'] = test_data_mean.apply(lambda x : x["Category"] + 1, axis = 1).astype('int') 
test_data_mean["ID"] = test_data_mean.index
test_data_group = test_data_mean[['ID','Category']]
print(test_data_group.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284 entries, 0 to 1283
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   ID        1284 non-null   int64
 1   Category  1284 non-null   int64
dtypes: int64(2)
memory usage: 30.1 KB
None
CPU times: user 112 ms, sys: 4 ms, total: 116 ms
Wall time: 119 ms


In [26]:
len(test_data_group.Category.unique())

43

In [29]:
submission_name = "submission_2.0_mean.csv"
test_data_group.to_csv(submission_name, index=False)

In [30]:
upload_files([submission_name])

Upload from local submission_2.0_mean.csv to data/breakfast-img-data/submission_2.0_mean.csv
